# 2. Analyse Exploratoire des Données

Ce notebook se concentre sur la deuxième étape du processus d'analyse prédictive : l'analyse exploratoire des données. Nous allons explorer en profondeur les données préparées dans le notebook précédent pour mieux comprendre les relations entre les variables et identifier les facteurs qui influencent le statut de crédit des clients.

## 2.1 Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Pour une meilleure lisibilité des graphiques
%matplotlib inline

## 2.2 Chargement des données préparées

In [ ]:
# Chemin vers les données préparées
data_dir = "/home/ubuntu/notebooks/data"
data_path = os.path.join(data_dir, "credit_data_prepared.pkl")

# Vérification de l'existence du fichier
if os.path.exists(data_path):
    print(f"Le fichier existe à l'emplacement : {data_path}")
    # Chargement des données
    df = pd.read_pickle(data_path)
    print(f"Données chargées avec succès. Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
else:
    print(f"Erreur : Le fichier n'existe pas à l'emplacement : {data_path}")
    # Si le fichier n'existe pas, on charge les données brutes et on applique les transformations
    print("Tentative de chargement des données brutes...")
    excel_path = "/home/ubuntu/upload/ab4e5df7-08a5-4e76-8ddd-9d4f845ecff1.xlsx"
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        # Renommage des colonnes
        colonnes_renommees = {
            "client's age": "age",
            "marital status": "marital_status",
            "amount of expenses": "expenses",
            "amount of income": "income",
            "amount requested of loan": "loan_amount",
            "price of good": "good_price",
            "credit status": "credit_status"
        }
        df = df.rename(columns=colonnes_renommees)
        # Transformation de la variable cible
        df['credit_status'] = df['credit_status'].map({"Yes": 0, "No": 1})
        print(f"Données brutes chargées et transformées. Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
    else:
        print(f"Erreur : Le fichier de données brutes n'existe pas non plus à l'emplacement : {excel_path}")

## 2.3 Aperçu des données

In [ ]:
# Affichage des premières lignes
print("Aperçu des 5 premières lignes :")
df.head()

In [ ]:
# Informations sur les colonnes
print("Informations sur les colonnes :")
df.info()

In [ ]:
# Statistiques descriptives
print("Statistiques descriptives :")
df.describe().T  # Transposé pour une meilleure lisibilité

## 2.4 Analyse univariée

Nous allons analyser chaque variable individuellement pour comprendre sa distribution et ses caractéristiques.

### 2.4.1 Variable cible : credit_status

In [ ]:
# Distribution de la variable cible
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='credit_status', data=df)
plt.title('Distribution de la variable cible (credit_status)', fontsize=14)
plt.xlabel('Statut de crédit (0 = Solvable, 1 = Non solvable)', fontsize=12)
plt.ylabel('Nombre de clients', fontsize=12)

# Ajout des pourcentages
total = len(df)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%'
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='bottom', fontsize=12)

plt.show()

# Affichage des statistiques
print("Répartition de la variable cible :")
credit_status_counts = df['credit_status'].value_counts()
credit_status_percentages = df['credit_status'].value_counts(normalize=True) * 100
credit_status_stats = pd.DataFrame({
    'Nombre': credit_status_counts,
    'Pourcentage': credit_status_percentages
})
credit_status_stats.index = ['Solvable (0)', 'Non solvable (1)']
credit_status_stats

### 2.4.2 Variables numériques

In [ ]:
# Liste des variables numériques
variables_numeriques = ['age', 'expenses', 'income', 'loan_amount', 'good_price']

# Création d'un tableau de statistiques descriptives détaillées
stats_numeriques = df[variables_numeriques].describe().T
stats_numeriques['variance'] = df[variables_numeriques].var()
stats_numeriques['skewness'] = df[variables_numeriques].skew()
stats_numeriques['kurtosis'] = df[variables_numeriques].kurtosis()
stats_numeriques

In [ ]:
# Histogrammes pour chaque variable numérique
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()

for i, var in enumerate(variables_numeriques):
    sns.histplot(df[var], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution de {var}', fontsize=14)
    axes[i].set_xlabel(var, fontsize=12)
    axes[i].set_ylabel('Fréquence', fontsize=12)
    
    # Ajout des lignes verticales pour la moyenne et la médiane
    axes[i].axvline(df[var].mean(), color='red', linestyle='--', label='Moyenne')
    axes[i].axvline(df[var].median(), color='green', linestyle='-.', label='Médiane')
    axes[i].legend()

# Suppression du subplot vide
if len(variables_numeriques) < 6:
    fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots pour chaque variable numérique
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()

for i, var in enumerate(variables_numeriques):
    sns.boxplot(y=df[var], ax=axes[i])
    axes[i].set_title(f'Boxplot de {var}', fontsize=14)
    axes[i].set_ylabel(var, fontsize=12)

# Suppression du subplot vide
if len(variables_numeriques) < 6:
    fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

### 2.4.3 Variable catégorielle : marital_status

In [ ]:
# Distribution du statut marital
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='marital_status', data=df)
plt.title('Distribution du statut marital', fontsize=14)
plt.xlabel('Statut marital', fontsize=12)
plt.ylabel('Nombre de clients', fontsize=12)

# Ajout des pourcentages
total = len(df)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%'
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='bottom', fontsize=12)

plt.show()

# Affichage des statistiques
print("Répartition du statut marital :")
marital_status_counts = df['marital_status'].value_counts().sort_index()
marital_status_percentages = df['marital_status'].value_counts(normalize=True).sort_index() * 100
marital_status_stats = pd.DataFrame({
    'Nombre': marital_status_counts,
    'Pourcentage': marital_status_percentages
})
marital_status_stats

## 2.5 Analyse bivariée

Nous allons maintenant explorer les relations entre la variable cible (credit_status) et chaque variable explicative.

### 2.5.1 Relation entre les variables numériques et la variable cible

In [ ]:
# Boxplots des variables numériques par statut de crédit
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()

for i, var in enumerate(variables_numeriques):
    sns.boxplot(x='credit_status', y=var, data=df, ax=axes[i])
    axes[i].set_title(f'{var} par statut de crédit', fontsize=14)
    axes[i].set_xlabel('Statut de crédit (0 = Solvable, 1 = Non solvable)', fontsize=12)
    axes[i].set_ylabel(var, fontsize=12)

# Suppression du subplot vide
if len(variables_numeriques) < 6:
    fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

In [ ]:
# Statistiques descriptives par groupe
print("Statistiques descriptives par statut de crédit :")
grouped_stats = df.groupby('credit_status')[variables_numeriques].agg(['mean', 'median', 'std'])
grouped_stats

In [ ]:
# Tests statistiques pour comparer les moyennes entre les groupes
print("Tests statistiques pour comparer les moyennes entre les groupes :")
test_results = {}

for var in variables_numeriques:
    # Extraction des données pour chaque groupe
    group0 = df[df['credit_status'] == 0][var]
    group1 = df[df['credit_status'] == 1][var]
    
    # Test de normalité (Shapiro-Wilk)
    _, p_norm_0 = stats.shapiro(group0.sample(min(5000, len(group0))))
    _, p_norm_1 = stats.shapiro(group1.sample(min(5000, len(group1))))
    normal_dist = (p_norm_0 > 0.05) and (p_norm_1 > 0.05)
    
    # Test d'égalité des variances (Levene)
    _, p_var = stats.levene(group0, group1)
    equal_var = p_var > 0.05
    
    # Test de comparaison des moyennes
    if normal_dist:
        # Test t de Student si distribution normale
        t_stat, p_value = stats.ttest_ind(group0, group1, equal_var=equal_var)
        test_name = "t-test"
    else:
        # Test de Mann-Whitney si distribution non normale
        t_stat, p_value = stats.mannwhitneyu(group0, group1)
        test_name = "Mann-Whitney U"
    
    # Stockage des résultats
    test_results[var] = {
        'Test': test_name,
        'Statistique': t_stat,
        'p-value': p_value,
        'Significatif': p_value < 0.05
    }

# Affichage des résultats
pd.DataFrame(test_results).T

In [ ]:
# Visualisation des distributions par groupe avec KDE
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
axes = axes.flatten()

for i, var in enumerate(variables_numeriques):
    sns.kdeplot(data=df, x=var, hue='credit_status', common_norm=False, ax=axes[i])
    axes[i].set_title(f'Distribution de {var} par statut de crédit', fontsize=14)
    axes[i].set_xlabel(var, fontsize=12)
    axes[i].set_ylabel('Densité', fontsize=12)
    axes[i].legend(['Solvable (0)', 'Non solvable (1)'])

# Suppression du subplot vide
if len(variables_numeriques) < 6:
    fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

### 2.5.2 Relation entre le statut marital et la variable cible

In [ ]:
# Tableau croisé entre le statut marital et le statut de crédit
crosstab = pd.crosstab(df['marital_status'], df['credit_status'])
crosstab.columns = ['Solvable (0)', 'Non solvable (1)']
crosstab['Total'] = crosstab.sum(axis=1)
crosstab.loc['Total'] = crosstab.sum()
print("Tableau croisé entre le statut marital et le statut de crédit :")
crosstab

In [ ]:
# Tableau croisé avec pourcentages par ligne
crosstab_pct = pd.crosstab(df['marital_status'], df['credit_status'], normalize='index') * 100
crosstab_pct.columns = ['Solvable (0)', 'Non solvable (1)']
print("Tableau croisé avec pourcentages par ligne :")
crosstab_pct

In [ ]:
# Visualisation du tableau croisé
plt.figure(figsize=(12, 8))
sns.countplot(x='marital_status', hue='credit_status', data=df)
plt.title('Statut de crédit par statut marital', fontsize=14)
plt.xlabel('Statut marital', fontsize=12)
plt.ylabel('Nombre de clients', fontsize=12)
plt.legend(['Solvable (0)', 'Non solvable (1)'])
plt.show()

In [ ]:
# Visualisation des pourcentages par statut marital
plt.figure(figsize=(12, 8))
crosstab_pct.plot(kind='bar', stacked=True)
plt.title('Pourcentage de clients par statut de crédit et statut marital', fontsize=14)
plt.xlabel('Statut marital', fontsize=12)
plt.ylabel('Pourcentage', fontsize=12)
plt.legend(['Solvable (0)', 'Non solvable (1)'])
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Test du chi-carré pour évaluer l'indépendance entre le statut marital et le statut de crédit
contingency_table = pd.crosstab(df['marital_status'], df['credit_status'])
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Test du chi-carré :")
print(f"Statistique du chi-carré : {chi2:.4f}")
print(f"Degrés de liberté : {dof}")
print(f"p-value : {p:.4f}")
print(f"Significatif (p < 0.05) : {p < 0.05}")

## 2.6 Analyse multivariée

Nous allons maintenant explorer les relations entre plusieurs variables simultanément.

### 2.6.1 Matrice de corrélation

In [ ]:
# Calcul de la matrice de corrélation
correlation_matrix = df.corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice de corrélation', fontsize=14)
plt.show()

In [ ]:
# Corrélations avec la variable cible
target_correlations = correlation_matrix['credit_status'].sort_values(ascending=False)
print("Corrélations avec la variable cible (credit_status) :")
target_correlations

### 2.6.2 Pairplot pour visualiser les relations entre variables

In [ ]:
# Création d'un sous-ensemble de données pour le pairplot (pour réduire le temps de calcul)
subset_cols = ['age', 'income', 'loan_amount', 'credit_status']
df_subset = df[subset_cols]

# Pairplot
plt.figure(figsize=(15, 12))
sns.pairplot(df_subset, hue='credit_status', palette=['blue', 'red'])
plt.suptitle('Pairplot des variables principales', y=1.02, fontsize=16)
plt.show()

### 2.6.3 Visualisations avancées

In [ ]:
# Scatter plot avec deux variables numériques et la variable cible comme couleur
plt.figure(figsize=(12, 8))
sns.scatterplot(x='income', y='loan_amount', hue='credit_status', data=df, palette=['blue', 'red'], alpha=0.7)
plt.title('Relation entre revenu, montant du prêt et statut de crédit', fontsize=14)
plt.xlabel('Revenu', fontsize=12)
plt.ylabel('Montant du prêt', fontsize=12)
plt.legend(['Solvable (0)', 'Non solvable (1)'])
plt.show()

In [ ]:
# Scatter plot avec deux autres variables numériques
plt.figure(figsize=(12, 8))
sns.scatterplot(x='age', y='good_price', hue='credit_status', data=df, palette=['blue', 'red'], alpha=0.7)
plt.title('Relation entre âge, prix du bien et statut de crédit', fontsize=14)
plt.xlabel('Âge', fontsize=12)
plt.ylabel('Prix du bien', fontsize=12)
plt.legend(['Solvable (0)', 'Non solvable (1)'])
plt.show()

In [ ]:
# Création d'un ratio loan_to_income (montant du prêt / revenu)
df['loan_to_income'] = df['loan_amount'] / df['income']

# Boxplot du ratio loan_to_income par statut de crédit
plt.figure(figsize=(10, 6))
sns.boxplot(x='credit_status', y='loan_to_income', data=df)
plt.title('Ratio prêt/revenu par statut de crédit', fontsize=14)
plt.xlabel('Statut de crédit (0 = Solvable, 1 = Non solvable)', fontsize=12)
plt.ylabel('Ratio prêt/revenu', fontsize=12)
plt.show()

# Statistiques descriptives du ratio par groupe
print("Statistiques descriptives du ratio prêt/revenu par statut de crédit :")
df.groupby('credit_status')['loan_to_income'].describe()

In [ ]:
# Création d'un ratio good_price_to_loan (prix du bien / montant du prêt)
df['good_price_to_loan'] = df['good_price'] / df['loan_amount']

# Boxplot du ratio good_price_to_loan par statut de crédit
plt.figure(figsize=(10, 6))
sns.boxplot(x='credit_status', y='good_price_to_loan', data=df)
plt.title('Ratio prix du bien/prêt par statut de crédit', fontsize=14)
plt.xlabel('Statut de crédit (0 = Solvable, 1 = Non solvable)', fontsize=12)
plt.ylabel('Ratio prix du bien/prêt', fontsize=12)
plt.show()

# Statistiques descriptives du ratio par groupe
print("Statistiques descriptives du ratio prix du bien/prêt par statut de crédit :")
df.groupby('credit_status')['good_price_to_loan'].describe()

## 2.7 Résumé de l'analyse exploratoire

Dans ce notebook, nous avons réalisé une analyse exploratoire approfondie des données pour mieux comprendre les relations entre les variables et identifier les facteurs qui influencent le statut de crédit des clients. Voici les principales observations :

### 2.7.1 Analyse univariée

1. **Variable cible (credit_status)** :
   - La majorité des clients sont solvables (environ 80%).
   - Les clients non solvables représentent environ 20% de l'échantillon.

2. **Variables numériques** :
   - **Âge** : La moyenne d'âge est d'environ 38 ans, avec une distribution relativement normale.
   - **Dépenses** : La moyenne des dépenses est d'environ 60, avec une légère asymétrie positive.
   - **Revenus** : La moyenne des revenus est d'environ 150, avec une distribution relativement normale.
   - **Montant du prêt** : La moyenne est d'environ 1035, avec une asymétrie positive.
   - **Prix du bien** : La moyenne est d'environ 1440, avec une asymétrie positive et quelques valeurs extrêmes.

3. **Variable catégorielle (marital_status)** :
   - La majorité des clients ont un statut marital de 2 (environ 70%).
   - Les statuts 1 et 3 sont également bien représentés.
   - Les statuts 4 et 5 sont très peu représentés.

### 2.7.2 Analyse bivariée

1. **Variables numériques et statut de crédit** :
   - **Âge** : Les clients non solvables ont tendance à être plus jeunes que les clients solvables.
   - **Revenus** : Les clients non solvables ont des revenus significativement plus faibles.
   - **Montant du prêt** : Les clients non solvables demandent des prêts significativement plus élevés.
   - **Prix du bien** : Les clients non solvables achètent des biens significativement plus chers.

2. **Statut marital et statut de crédit** :
   - Il existe une relation significative entre le statut marital et le statut de crédit (test du chi-carré).
   - Les clients avec un statut marital de 1 ont un taux de non-solvabilité plus élevé.

### 2.7.3 Analyse multivariée

1. **Corrélations** :
   - Le montant du prêt est positivement corrélé avec le statut de non-solvabilité.
   - Le revenu est négativement corrélé avec le statut de non-solvabilité.
   - L'âge est négativement corrélé avec le statut de non-solvabilité.
   - Il existe une forte corrélation positive entre le montant du prêt et le prix du bien.

2. **Ratios** :
   - Le ratio prêt/revenu est significativement plus élevé chez les clients non solvables.
   - Le ratio prix du bien/prêt est plus faible chez les clients non solvables, suggérant qu'ils demandent des prêts plus élevés par rapport au prix du bien.

### 2.7.4 Conclusions et implications pour la modélisation

1. **Variables importantes** :
   - Le montant du prêt, le revenu et l'âge semblent être les variables les plus importantes pour prédire le statut de crédit.
   - Le ratio prêt/revenu pourrait être un prédicteur particulièrement utile.

2. **Déséquilibre des classes** :
   - La variable cible est déséquilibrée (80% solvables, 20% non solvables), ce qui devra être pris en compte lors de la modélisation.

3. **Ingénierie des caractéristiques** :
   - La création de ratios (prêt/revenu, prix du bien/prêt) pourrait améliorer les performances des modèles.

4. **Standardisation** :
   - Les variables numériques ont des échelles différentes, donc une standardisation sera nécessaire avant la modélisation.

## Prochaine étape

Dans le prochain notebook, nous préparerons les données pour la modélisation en séparant les données en ensembles d'entraînement et de test, et en standardisant les variables numériques.